In [1]:
import sqlite3 as sq
import pandas as pd
import csv

In [2]:
#1. Создайте и заполните таблицу запросами, 
#создайте техническое поле ID c параметрами 
#INTEGER PRIMARY KEY AUTOINCREMENT.

In [3]:
connection = sq.connect("training.db")
cursor = connection.cursor()
cursor.execute("""CREATE TABLE works (
ID INTEGER PRIMARY KEY AUTOINCREMENT,
salary MONEY,
educationType VARCHAR(100),
jobTitle VARCHAR(100),
qualification VARCHAR(100),
gender VARCHAR(10),
dateModify DATE,
skills VARCHAR(1000),
otherInfo VARCHAR(1000))""")

In [9]:
with open('works.csv', 'r', encoding='UTF-8') as works:
    reader = csv.DictReader(works)
    to = ([(i['salary'],
            i['educationType'],
            i['jobTitle'],
            i['qualification'],
            i['gender'],
            i['dateModify'],
            i['skills'],
            i['otherInfo']) for i in reader])
    
cursor.executemany("""INSERT INTO works
    (salary, educationType, jobTitle, qualification, gender,
    dateModify, skills, otherInfo)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", to)

In [10]:
#2. Добавьте индекс на поле salary.
#Изменится ли после этого размер файла? На сколько?

In [11]:
import os
print("Размер файла до добавления индекса: " 
      + str(os.stat("training.db").st_size))

Размер файла до добавления индекса: 4747264


In [12]:
cursor.execute("CREATE INDEX salary_index ON works (salary)")

In [13]:
print("Размер файла после добавления индекса: " 
      + str(os.stat("training.db").st_size))

Размер файла после добавления индекса: 6029312


In [14]:
#3. Выведите количество записей

In [21]:
cursor.execute("SELECT COUNT(*) FROM works")
print("Количество записей в таблице: " + str(cursor.fetchone())[1:-2:])

Количество записей в таблице: 32683


In [ ]:
#4. Выведите количество мужчин и женщин

In [22]:
cursor.execute("SELECT COUNT(*) FROM works WHERE gender = 'Мужской'")
print("Количество мужчин в таблице: " + str(cursor.fetchone())[1:-2:])

cursor.execute("SELECT COUNT(*) FROM works WHERE gender = 'Женский'")
print("Количество женщин в таблице: " + str(cursor.fetchone())[1:-2:])

Количество мужчин в таблице: 13386
Количество женщин в таблице: 17910


In [ ]:
#5. У скольки записей заполены skills?

In [23]:
cursor.execute("SELECT COUNT(*) FROM works WHERE skills != ''")
print("Количество записей с заполненным полем skills: " + str(cursor.fetchone())[1:-2:])

Количество записей с заполненным полем skills: 8972


In [ ]:
#6. Получить заполненные скиллы

In [25]:
cursor.execute("SELECT skills FROM works WHERE skills != ''")
cursor.fetchall()

[('<p>Аналитическое&nbsp;мышление,&nbsp;<span class="s6"><span class="bumpedFont15">ответственность, </span></span><span class="s6"><span class="bumpedFont15">стабильность психологического состояния и настроения.&nbsp;</span></span></p>',),
 ('Ответственная,пунктуальная,дисциплинированная я,стрессоустойчивая,легкообучаема ,технически оснащена',),
 ('<p>Коммуникабельность&nbsp;</p>',),
 ('<p>Ответственность в работе</p>',),
 ('<p>Усидчивость, умение удерживать в памяти нужную информацию,скрупулезность, пунктуальность, умение планировать и организовать рабочий процесс,дальновидность, умение собирать и анализировать данные, аккуратность, нацеленность на результат.</p>',),
 ('<p>Владение&nbsp;ПК:&nbsp;MsOffice,&nbsp;1С:ЗУП&nbsp;8.3,&nbsp;&laquo;Консультант+&raquo;</p><p>Знание&nbsp;ТК&nbsp;РФ,&nbsp;делового&nbsp;этикета.</p><p>Опыт&nbsp;работы&nbsp;в&nbsp;условиях&nbsp;многозадачности.</p><p>Опыт&nbsp;консультирования&nbsp;клиентов.&nbsp;</p><p>Консультирование&nbsp;сотрудников&nbsp;по&nbs

In [ ]:
#7. Вывести зарплату только у тех, у кого в скилах есть Python.

In [33]:
cursor.execute("SELECT salary FROM works WHERE skills LIKE '%Python%'")
for s in cursor.fetchall():
    print(str(s)[1:-2], end=', ')

35000, 20000, 35000, 15000, 25000, 90000, 30000, 50000, 23000, 23000, 

In [ ]:
#8. Создайте отдельную таблицу с гендером, заполните ее значениями,
#сделайте на нее внешний ключ из таблицы works.

In [34]:
cursor.execute("""CREATE TABLE gender (
    ID INTEGER PRIMARY KEY NOT NULL,
    NAME CHAR(15))""")

cursor.execute("""INSERT INTO gender VALUES
    (1, 'Мужской'),
    (2, 'Женский')""")

In [ ]:
#9. Отдельная таблица для образования.

In [35]:
cursor.execute("""CREATE TABLE education (
    ID INTEGER PRIMARY KEY,
    NAME VARCHAR(100))""")

cursor.execute("""INSERT INTO education VALUES
    (1, 'Среднее'),
    (2, 'Среднее профессиональное'),
    (3, 'Незаконченное высшее'),
    (4, 'Высшее')""")

In [ ]:
#10. Отдельная таблица для jobTitle. 
#Попытайтесь свести все разнообразие к небольшому количеству должностей.

In [36]:
cursor.execute("""CREATE TABLE job (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NAME VARCHAR(100))""")

jobs = ([(i + 1, item[0].lower()) for i, item in
             enumerate(cursor.execute("""SELECT DISTINCT jobTitle FROM works
             WHERE jobTitle != '' AND jobTitle NOT LIKE '%.%' AND jobTitle NOT LIKE '%.%'""").fetchall())])
cursor.executemany("INSERT INTO job VALUES (?, ?)", jobs)

In [40]:
cursor.execute("""CREATE TABLE qualification (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NAME VARCHAR(100))""")

qualifications = ([(i + 1, item[0].lower()) for i, item in
             enumerate(cursor.execute("""SELECT DISTINCT jobTitle FROM works
             WHERE qualification != '' AND qualification NOT LIKE '%.%' AND qualification NOT LIKE '%.%'""").fetchall())])
cursor.executemany("INSERT INTO qualification VALUES (?, ?)", qualifications)